In [1]:
#Se crea la conexion a la base de datos utilizando el patron Singleton para garantizar una unica conexion y se realizan todas las importaciones necesarias

import sys
import os

sys.path.append(os.path.abspath(os.path.join('..', 'Avance2y3/tests')))

from patterns.singleton_SQL import MySQLConnection
from patterns.builder_SQL import QueryBuilder
from patterns.strategy_SQL import KPIContext, TotalSalesKPI, SalesByCategoryKPI, CustomersCountKPI
from test_patterns import run_all_tests

conn1 = MySQLConnection()


Iniciada la conexion a la base


In [2]:
#Query basica usando el patron Singleton

query = """
        SELECT * FROM EMPLOYEES
        """
df = conn1.execute_query(query)
print(df)

    EmployeeID  FirstName MiddleInital   LastName  BirthDate Gender  CityID  \
0            1     Nicole            T     Fuller 1981-03-07      F      80   
1            2  Christine            W     Palmer 1968-01-25      F       4   
2            3      Pablo            Y      Cline 1963-02-09      M      70   
3            4    Darnell            O    Nielsen 1989-02-06      M      39   
4            5    Desiree            L     Stuart 1963-05-03      F      23   
5            6      Holly            E    Collins 1987-01-13      M      65   
6            7   Chadwick            P       Cook 1970-05-02      M      39   
7            8      Julie            E       Dyer 1956-12-13      M      18   
8            9     Daphne            X       King 1956-05-02      F      39   
9           10       Jean            P       Vang 1963-12-30      M       9   
10          11      Sonya            E    Dickson 1976-01-14      F      12   
11          12    Lindsay            M       Chen 19

In [3]:
#Se utiliza el patron Builder para ir armando dinamicamente una consulta
builder = QueryBuilder()
query = (builder.select("c.CategoryName", "COUNT(s.TotalPrice) AS total_sales")
               .from_table("sales s")
               .join("JOIN products p ON s.ProductID = p.ProductID")
               .join("JOIN categories c ON p.CategoryId = c.CategoryId")
               .group_by("c.CategoryName")
               .order_by("total_sales DESC")
               .limit(3)
               .build())
print("Consulta construida con Builder:")
print(query)

df_builder = conn1.execute_query(query)
print("\nResultado de la consulta Builder:")
print(df_builder)

Consulta construida con Builder:
SELECT c.CategoryName, COUNT(s.TotalPrice) AS total_sales FROM sales s JOIN products p ON s.ProductID = p.ProductID JOIN categories c ON p.CategoryId = c.CategoryId GROUP BY c.CategoryName ORDER BY total_sales DESC LIMIT 3

Resultado de la consulta Builder:
  CategoryName  total_sales
0  Confections         6299
1         Meat         5486
2      Poultry         5156


In [4]:
#Se utiliza el patron Strategy para armar distintos KPIs

kpi_context = KPIContext(TotalSalesKPI(conn1))
total_sales = kpi_context.get_kpi()
print(f"Total ventas: ${total_sales}")

kpi_context = KPIContext(CustomersCountKPI(conn1))
total_customers = kpi_context.get_kpi()
print(f"Total customers: {total_customers}")

kpi_context = KPIContext(SalesByCategoryKPI(conn1))
sales_category = kpi_context.get_kpi()
print("Sales by category:")
print(f"{sales_category}")

Total ventas: $7863619.00
Total customers: 98759
Sales by category:
   CategoryName  total_sales
0   Confections         6299
1          Meat         5486
2       Poultry         5156
3       Cereals         5110
4       Produce         4771
5     Beverages         4196
6        Snails         4119
7       Seafood         4059
8    Shell fish         3949
9         Dairy         3892
10        Grain         2963


In [5]:
#Se corre esta funcion general de test que lo que hace es correr todos los tests para el avance 2
run_all_tests()

✔ test_singleton_connection passed
✔ test_builder_query_structure passed
✔ test_strategy_total_sales_kpi passed
✔ test_strategy_sales_by_category_kpi passed
✔ test_strategy_customers_count_kpi passed
✅ Todas las pruebas pasaron correctamente.


In [ ]:
################################ AVANCE 3 ##########################################

#las justificaciones de las decisiones se encuentran en el README

In [ ]:
#Creacion de indices desde python

# Índice en SalesPersonID (para uniones con la tabla employees)
conn1.execute_ddl("CREATE INDEX idx_sales_salespersonid ON sales (SalesPersonID);")

#Índice en CustomerId (para uniones con la tabla customers)
conn1.execute_ddl("CREATE INDEX idx_sales_customerid ON sales (CustomerId);")

#Verifico los indices si se crearon
verify_indexs = conn1.execute_query("SHOW INDEX FROM sales;")
print(verify_indexs)

Sentencia DDL ejecutada exitosamente.
Sentencia DDL ejecutada exitosamente.
   Table  Non_unique                 Key_name  Seq_in_index    Column_name  \
0  sales           0                  PRIMARY             1        SalesID   
1  sales           1  idx_sales_salespersonid             1  SalesPersonID   
2  sales           1     idx_sales_customerid             1     CustomerId   

  Collation  Cardinality Sub_part Packed Null Index_type Comment  \
0         A        52607     None   None           BTREE           
1         A           22     None   None  YES      BTREE           
2         A        39494     None   None  YES      BTREE           

  Index_comment Visible Expression  
0                   YES       None  
1                   YES       None  
2                   YES       None  


In [7]:
#CTE para obtener el total de ventas por producto, y luego filtrar los productos que hayan tenido más de 100 ventas.

query = """
        WITH ventas_por_producto AS (
            SELECT 
                ProductID,
                COUNT(*) AS total_ventas
            FROM sales
            GROUP BY ProductID
        )
        SELECT 
            vp.ProductID,
            vp.total_ventas,
            p.ProductName
        FROM ventas_por_producto vp
        JOIN products p ON p.ProductID = vp.ProductID
        WHERE vp.total_ventas > 100;
        """
df = conn1.execute_query(query)
print(df)

     ProductID  total_ventas                      ProductName
0            1           108              Flour - Whole Wheat
1            2           114       Cookie Chocolate Chip With
2            3           111               Onions - Cippolini
3            4           123       Sauce - Gravy; Au Jus; Mix
4            5           101           Artichokes - Jerusalem
..         ...           ...                              ...
371        446           101               Sunflower Seed Raw
372        447           102       Salmon - Atlantic; Skin On
373        450           132    Wine - Vidal Icewine Magnotta
374        451           125  Soup - Campbells Tomato Ravioli
375        452           122          Napkin White - Starched

[376 rows x 3 columns]


In [8]:
#Funcion ventana para cada venta, mostrar el total acumulado de ventas por cliente (customer_id) ordenado cronológicamente.

query = """
        SELECT
        CustomerId,
        SalesDate,
        Quantity,
        SUM(Quantity) OVER (PARTITION BY CustomerId ORDER BY SalesDate) AS acumulado_cliente
        FROM sales; 
        """
df = conn1.execute_query(query)
print(df)

       CustomerId       SalesDate  Quantity acumulado_cliente
0               1 0 days 23:37:01         1                 1
1               1 2 days 07:10:00         1                 2
2               4 1 days 04:56:00         1                 1
3               8 0 days 11:43:00         1                 1
4              10 0 days 15:16:00         1                 1
...           ...             ...       ...               ...
49995       98749 1 days 09:20:00        25                25
49996       98750 1 days 14:20:01        25                25
49997       98751 1 days 06:33:00        25                25
49998       98752 1 days 19:02:00        25                25
49999       98756 2 days 09:40:00        25                25

[50000 rows x 4 columns]


In [9]:
#Se crea y se ejecuta una vista desde python 
view_sql = """
        CREATE OR REPLACE VIEW detailed_sales_report AS
        SELECT
            s.SalesID,
            s.Quantity,
            s.Discount,
            s.TotalPrice,
            s.SalesDate,
            s.TransactionNumber,
            p.ProductName,
            p.Price AS ProductUnitPrice,
            c.CategoryName,
            p.Class,
            p.Resistant,
            p.IsAllergic,
            p.VitalityDays,
            cust.FirstName AS CustomerFirstName,
            cust.LastName AS CustomerLastName,
            city.CityName AS CustomerCity,
            country.CountryName AS CustomerCountry,
            emp.FirstName AS SalesPersonFirstName,
            emp.LastName AS SalesPersonLastName,
            emp.Gender AS SalesPersonGender
        FROM
            sales s
        JOIN
            products p ON s.ProductID = p.ProductID
        JOIN
            categories c ON p.CategoryID = c.CategoryID
        JOIN
            customers cust ON s.CustomerId = cust.CustomerID
        JOIN
            cities city ON cust.CityID = city.CityID
        JOIN
            countries country ON city.CountryID = country.CountryID
        JOIN
            employees emp ON s.SalesPersonID = emp.EmployeeID;
        """
conn1.execute_ddl(view_sql)
print("Vista 'detailed_sales_report' creada.")

query_view_sql = "SELECT * FROM detailed_sales_report LIMIT 5;"
df_sales_report = conn1.execute_query(query_view_sql)
print(df_sales_report)

Sentencia DDL ejecutada exitosamente.
Vista 'detailed_sales_report' creada.
   SalesID  Quantity Discount TotalPrice       SalesDate  \
0     1379        14     0.10      14.00 1 days 00:52:01   
1     7884        11     0.00      11.00 1 days 14:52:01   
2    10389        22     0.00      22.00 1 days 23:34:00   
3    10628        22     0.00      22.00 0 days 22:36:00   
4    13687        13     0.00      13.00 1 days 16:58:01   

        TransactionNumber                 ProductName ProductUnitPrice  \
0  7GWH7VPLA9Y79GNXZ4ZS\r   Ice Cream Bar - Drumstick          24.7560   
1  OZCGXCI0K7XTJOBQR2QH\r                 Scampi Tail          95.0957   
2  1YL3MRV7RLUDEV0MSFPD\r  Wine - Sogrape Mateus Rose           3.7805   
3  GNDKLBFCE4KONCOTJR55\r               Vanilla Beans          97.8435   
4  W1T93XHO8LUHX2NAOYN1\r                 Bread - Rye          83.5373   

  CategoryName   Class Resistant IsAllergic  VitalityDays CustomerFirstName  \
0        Dairy  Medium      Weak       

In [10]:
#Se crea una funcion desde python 
function_sql = """
                DELIMITER $$

                CREATE FUNCTION CALCULATE_SALE_TOTAL_PRICE(
                    p_unit_price DECIMAL(10,4),
                    p_quantity INT,
                    p_discount DECIMAL(4,2)
                )
                RETURNS DECIMAL(10,2)
                DETERMINISTIC
                BEGIN
                    DECLARE v_total_price DECIMAL(10,2);

                    -- Validar que el descuento no sea negativo y no supere el 100%
                    IF p_discount < 0 OR p_discount > 1 THEN
                        -- Puedes manejar este error de alguna manera, por ejemplo, lanzando una excepción
                        -- Para este ejemplo, simplemente ajustaremos el descuento si está fuera de rango.
                        SET p_discount = LEAST(GREATEST(p_discount, 0), 1); -- Asegura que esté entre 0 y 1
                    END IF;

                    -- Calcular el precio total: (precio_unitario * cantidad) * (1 - descuento)
                    SET v_total_price = (p_unit_price * p_quantity) * (1 - p_discount);

                    RETURN v_total_price;
                END $$

                DELIMITER ;
                """
conn1.execute_ddl(function_sql)
print("Funcion 'CALCULATE_SALE_TOTAL_PRICE' creada.")




Sentencia DDL ejecutada exitosamente.
Funcion 'CALCULATE_SALE_TOTAL_PRICE' creada.


In [11]:
#Se ejecuta la funcion desde python 

# Ejemplos de uso de la función
print("\nEjemplos de uso de la función 'CALCULATE_SALE_TOTAL_PRICE':")       
# Ejemplo 1: Sin descuento
df1 = conn1.execute_query("SELECT CALCULATE_SALE_TOTAL_PRICE(100.00, 2, 0.00) AS TotalPrice;")
print(f"Precio Total (100 * 2, sin descuento): {df1.iloc[0]['TotalPrice']}")
# Ejemplo 2: Con 10% de descuento
df2 = conn1.execute_query("SELECT CALCULATE_SALE_TOTAL_PRICE(100.00, 2, 0.10) AS TotalPrice;")
print(f"Precio Total (100 * 2, 10% descuento): {df2.iloc[0]['TotalPrice']}")
# Ejemplo 3: Con 25% de descuento y cantidad diferente
df3 = conn1.execute_query("SELECT CALCULATE_SALE_TOTAL_PRICE(75.50, 5, 0.25) AS TotalPrice;")
print(f"Precio Total (75.50 * 5, 25% descuento): {df3.iloc[0]['TotalPrice']}")
# Ejemplo 4: Usar la función con datos las tablas (ej: para un producto específico)
print("\nCalculando el precio de una venta usando datos de la tabla products (ProductoID = 1):")
query_with_data = """
SELECT
    p.ProductName,
    p.Price,
    5 AS QuantitySold, -- Cantidad arbitraria para el ejemplo
    0.15 AS DiscountApplied, -- Descuento arbitrario
    CALCULATE_SALE_TOTAL_PRICE(p.Price, 5, 0.15) AS CalculatedSalePrice
FROM
    products p
WHERE
    p.ProductID = 1;
"""
df_product_sale = conn1.execute_query(query_with_data)
print(df_product_sale)


Ejemplos de uso de la función 'CALCULATE_SALE_TOTAL_PRICE':
Precio Total (100 * 2, sin descuento): 200.00
Precio Total (100 * 2, 10% descuento): 180.00
Precio Total (75.50 * 5, 25% descuento): 283.13

Calculando el precio de una venta usando datos de la tabla products (ProductoID = 1):
           ProductName    Price  QuantitySold DiscountApplied  \
0  Flour - Whole Wheat  74.2988             5            0.15   

  CalculatedSalePrice  
0              315.77  


In [12]:
conn1.close_engine()

Finalizada la conexion a la base
